### Mastermind Final Project
By Francis Ng

Note: all code here is approximately the same as the python documents so you can run it anywhere you want. The only differences are this document comes with more explicit instructions, some code is changed to fit the style of a jupyter notebook, and the Solve class doesn't have the multisolveAll class (which was mainly used to get results)

In [1]:
import numpy as np
import random as rand

### Game Class
This is the actual mastermind game itself. I initially called it "Game" generically because I wanted to be able to build other games on top of this module, but ran out of time.

This part of the code isn't as important, but it essentially allows you to play mastermind! To give it a shot, simply create a game like Game() to start a default game. You can specify how many colors, guesses, and pegs you will play with total.

For each game object, simply input game.guess(your_guess_as_a_tuple) to guess and it'll display the board back to you. The board stays updated so don't worry about losing it. If you need, you can use game.cheat() to sneakily find the answer to the board.

In [2]:
import random as rand
import numpy as np

class Game:
	def __init__(self, code = None, colors=6, pegs=4, total_guesses=10, silent=False):
		if not silent:
			print("Starting new game with", colors, "colors,", pegs," pegs, and ", total_guesses, "total guesses allowed")
		if code == None:
			self.answer = []
			for i in range(pegs):
				self.answer.append(rand.randrange(colors))
		else:
			assert len(code) == pegs, "Code incorrect length!"
			self.answer = code

		self.board = []
		self.guesses = []
		self.solved = False
		self.failed = False
		self.silent = silent
		self.pegs = pegs
		self.colors = colors

		self.total_guesses = total_guesses

	def guess(self, *guesses):
		if self.solved:
			print("Already solved!")
			return
		elif self.failed:
			print("Sorry, you lost :(")
			return
		assert self.pegs == len(guesses)
		total_guesses = self.total_guesses
		xs = 0
		os = 0

		leftover_c = []
		leftover_g = []
		for i in range(self.pegs):
			if self.answer[i] == guesses[i]:
				os = os+1
			else:
				leftover_c.append(self.answer[i])
				leftover_g.append(guesses[i])
		for g in leftover_g:
			exists = False
			for c in leftover_c:
				if g == c:
					exists = True
			if exists:
				xs = xs+1
				leftover_c.remove(c)

		assert xs+os < (self.pegs+1), "What happened??"
		roundguess = []
		for i in range(os):
			roundguess.append("o")
		for i in range(xs):
			roundguess.append("x")
		for i in range(self.pegs-os-xs):
			roundguess.append("_")
		self.guesses.append(guesses)
		self.board.append(roundguess)

		if not self.silent:
			self.display_board()

		win = True
		for i in roundguess:
			if i != "o":
				win = False
		if win:
			if not self.silent:
				print("Congrats you win!")
			self.solved = True
		elif len(self.guesses) == self.total_guesses:
			if not self.silent:
				print("Game over, the correct answer was ", self.answer)
			self.failed = True

		return os, xs

	def rounds(self):
		return len(self.guesses)

	def cheat(self):
		return self.answer

	def display_board(self):
		print("Board:")
		for i in np.arange(self.total_guesses):
			if i <= (len(self.board)-1):
				start = "|"
				for z in range(self.pegs):
					start = start + " " + str(self.guesses[i][z])
				start = start + " |"
				for z in range(self.pegs):
					start = start + " " + str(self.board[i][z])
				start = start + " |"
				print(start)
			else:
				start = "|"
				for z in range(self.pegs):
					start = start + " _ "
				start = start + " |"
				for z in range(self.pegs):
					start = start + " _ "
				start = start + " |"
				print(start)

### Solver Class
Note: Hardcoded varying peg numbers

To use: specify how many colors (infinite amounts) and pegs (ranging from 4 to 6) you want to use as Solver(pegs=x, colors=y).
To use the solver, simply use the Solver.solve(game) method and feed it a "Game" class object to have it solve it! More details on how it solves in writeup.

Use Solver.multisolve(x) to have the solver take on x number of games for default settings (can go upwards of 10,000 at a decent speed but not very optimized so it's not recommended to go higher unless you have time). Any higher amount of games / pegs / colors will take a long time so please use with discretion.

For solve and multisolve, there are counterparts solveF and multisolverF, which also take in a first guess for every attempt! Try it out with the first guess as [0, 0, 1, 1]!

In [14]:
class Solver:
	def __init__(self, pegs=4, colors=6):
		hypotheses = []
		if pegs == 4:
			for i in range(colors):
				for j in range(colors):
					for k in range(colors):
						for l in range(colors):
							hypotheses.append(([i, j, k, l], 1/(pow(4, colors))))
		elif pegs == 5:
			for i in range(colors):
				for j in range(colors):
					for k in range(colors):
						for l in range(colors):
							for h in range(colors):
								hypotheses.append(([i, j, k, l, h], 1/(pow(5, colors))))
		elif pegs == 6:
			for i in range(colors):
				for j in range(colors):
					for k in range(colors):
						for l in range(colors):
							for h in range(colors):
								for y in range(colors):
									hypotheses.append(([i, j, k, l, h, y], 1/(pow(6, colors))))
		self.pegs = pegs
		self.hypotheses = hypotheses
		self.colors = colors

	def guess(self, game, red=1/4, white=3/16, force=None):
		assert self.pegs == game.pegs, "Different amount of pegs"
		assert self.colors == game.colors, "Different amount of colors"
		max_prob = 0
		max_hypotheses = []
		for h, p in self.hypotheses:
			if max_prob < p:
				max_prob = p
				max_hypotheses = [h]
			elif max_prob == p:
				max_hypotheses.append(h)
		if force == None:
			choice = rand.choice(max_hypotheses) #chooses one of the choices with the highest hypotheses chance
		else:
			choice = force #Forces a specific guess in the form of a list
		os, xs = game.guess(*choice)
		new_hypotheses = []
		for h, chance in self.hypotheses:
			oc = 0
			xc = 0
			leftover_c = []
			leftover_g = []

			if choice == h:
				new_hypotheses = new_hypotheses #filler
			else:
				for i in range(len(choice)):
					if h[i] == choice[i]:
						oc = oc+1
					else:
						leftover_c.append(h[i])
						leftover_g.append(choice[i])

				for g in leftover_g:
					exists = False
					for c in leftover_c:
						if g == c:
							exists = True
					if exists:
						xc = xc+1
						leftover_c.remove(c)
				if oc == os and xs == xc:
					new_hypotheses.append((h, ((oc*red) + (xs*white))))#Red and White are changeable parameters
		self.hypotheses = new_hypotheses

	def reset(self):
		colors = self.colors
		pegs = self.pegs
		hypotheses = []
		if pegs == 4:
			for i in range(colors):
				for j in range(colors):
					for k in range(colors):
						for l in range(colors):
							hypotheses.append(([i, j, k, l], 1/(pow(4, colors))))
		elif pegs == 5:
			for i in range(colors):
				for j in range(colors):
					for k in range(colors):
						for l in range(colors):
							for h in range(colors):
								hypotheses.append(([i, j, k, l, h], 1/(pow(5, colors))))
		elif pegs == 6:
			for i in range(colors):
				for j in range(colors):
					for k in range(colors):
						for l in range(colors):
							for h in range(colors):
								for y in range(colors):
									hypotheses.append(([i, j, k, l, h, y], 1/(pow(6, colors))))
		self.hypotheses = hypotheses

	def solve(self, game, hush=False): #Basic Solving Technique
		self.reset()
		while game.solved == False and game.failed == False:
			self.guess(game)
		if game.solved==True:
			if not hush:	
				print("Solved in", game.rounds(), "turns.")
			return True, game
		else:
			if not hush:
				print("Whoops we lost")
			return False, game

	def solveF(self, game, firstGuess, hush=False): #Basic Solving Technique
		self.reset()
		self.guess(game, force=firstGuess)
		while game.solved == False and game.failed == False:
			self.guess(game)
		if game.solved==True:
			if not hush:	
				print("Solved in", game.rounds(), "turns.")
			return True, game
		else:
			if not hush:
				print("Whoops we lost")
			return False, game

	def multisolve(self, games):
		wins = []
		losses = 0
		codes = []
		most = 0
		least = 10
		for i in range(games):
			g = Game(colors = self.colors, pegs=self.pegs, silent=True)
			win, game = self.solve(g, hush=True)
			guesses = game.rounds()
			answer = game.cheat()
			if guesses < least:
				least = guesses
				least_g = answer
			if guesses > most:
				most = guesses
				most_g = answer
			if win:
				wins.append(guesses)
			else:
				game.display_board()
				losses = losses + 1
				codes.append(answer)

		print("Won", len(wins), "number of times with an average of: ", sum(wins)/len(wins), "rounds played.")
		print("The game that took the most guesses was", most, "with the code", most_g)
		print("The game that took the least guesses was", least, "with the code", least_g)
		print("Lost", losses, "number of times to these codes:", codes)
		return codes

	def multisolveF(self, firstGuess, games):
		wins = []
		losses = 0
		codes = []
		most = 0
		least = 10
		for i in range(games):
			g = Game(colors = self.colors, pegs=self.pegs, silent=True)
			win, game = self.solveF(g, firstGuess, hush=True)
			guesses = game.rounds()
			answer = game.cheat()
			if guesses < least:
				least = guesses
				least_g = answer
			if guesses > most:
				most = guesses
				most_g = answer
			if win:
				wins.append(guesses)
			else:
				game.display_board()
				losses = losses + 1
				codes.append(answer)

		print("Won", len(wins), "number of times with an average of: ", sum(wins)/len(wins), "rounds played.")
		print("The game that took the most guesses was", most, "with the code", most_g)
		print("The game that took the least guesses was", least, "with the code", least_g)
		print("Lost", losses, "number of times to these codes:", codes)
		return codes

### Testing
Feel free to test out any part of the code here. A template of how everything runs is provided.

In [4]:
g1 = Game(code=[4, 2, 3, 5])

Starting new game with 6 colors, 4  pegs, and  10 total guesses allowed


In [15]:
s = Solver() #default solver

In [6]:
g1.guess(0, 0, 1, 1)
g1.guess(2, 2, 3, 3)
g1.guess(4, 4, 5, 5)

Board:
| 0 0 1 1 | _ _ _ _ |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
Board:
| 0 0 1 1 | _ _ _ _ |
| 2 2 3 3 | o o _ _ |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
Board:
| 0 0 1 1 | _ _ _ _ |
| 2 2 3 3 | o o _ _ |
| 4 4 5 5 | o o _ _ |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |


(2, 0)

In [10]:
g1.guess(4, 2, 3, 5)

Board:
| 0 0 1 1 | _ _ _ _ |
| 2 2 3 3 | o o _ _ |
| 4 4 5 5 | o o _ _ |
| 4 4 3 3 | o o _ _ |
| 4 4 3 5 | o o o _ |
| 4 3 3 5 | o o o _ |
| 4 2 3 5 | o o o o |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
Congrats you win!


(4, 0)

In [11]:
g_solve = Game(code=[4, 2, 3, 5])

Starting new game with 6 colors, 4  pegs, and  10 total guesses allowed


In [12]:
s.solve(g_solve)

Board:
| 1 5 2 0 | x x _ _ |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
Board:
| 1 5 2 0 | x x _ _ |
| 2 2 0 5 | o o _ _ |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
Board:
| 1 5 2 0 | x x _ _ |
| 2 2 0 5 | o o _ _ |
| 0 2 0 3 | o x _ _ |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
| _  _  _  _  | _  _  _  _  |
Board:
| 1 5 2 0 | x x _ _ |
| 2 2 0 5 | o o _ _ |
| 0 2 0 3 | o x _ _ |
| 2 4 0 1 | x x _ _ |
| _  _  _  _  | _  _  _  _  |
| 

(True, <__main__.Game at 0x7fbb3deb2208>)

In [18]:
s.multisolve(10000) #typically gives an average of 4.7

Won 10000 number of times with an average of:  4.7185 rounds played.
The game that took the most guesses was 8 with the code [2, 1, 4, 5]
The game that took the least guesses was 1 with the code [2, 2, 1, 0]
Lost 0 number of times to these codes: []


[]

In [19]:
s.multisolveF([0, 0, 1, 1], 10000) #typically gives a slightly lower average (0.03 consistently) but barely

Won 10000 number of times with an average of:  4.6749 rounds played.
The game that took the most guesses was 8 with the code [5, 4, 5, 2]
The game that took the least guesses was 1 with the code [0, 0, 1, 1]
Lost 0 number of times to these codes: []


[]